In [11]:
import elephant_rumble_inference as eri
import torch
import time

In [7]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

aves_hubert_model          = eri.AvesTorchaudioWrapper().to(DEVICE)
elephant_rumble_classifier = eri.ElephantRumbleClassifier().to('cpu')
raven_file_helper          = eri.RavenFileHelper("./data/Rumble")
audio_file_visualizer      = eri.AudioFileVisualizer()
audio_file_processor       = eri.AudioFileProcessor(aves_hubert_model,elephant_rumble_classifier,device=DEVICE)

elephant_rumble_classifier.load_pretrained_weights('best')

# This notebook won't train, only run inferrence. 
elephant_rumble_classifier.eval()
aves_hubert_model.eval()


using best.pth


In [55]:
test_files = raven_file_helper.get_files_from_test_folder()
audio_file = test_files[0]
audio_file = 'nn08c_20200104_000000.wav'

In [56]:

audio_path = raven_file_helper.audio_filename_to_path[audio_file]


In [57]:
scores = audio_file_processor.classify_wave_file_for_rumbles(audio_path, limit_audio_hours=24)


Classifying hour 0 of ./data/Rumble/Testing/PNNN/Sounds/nn08c_20200104_000000.wav torch.Size([0, 1]), torch.Size([1800000, 1]), torch.Size([5120, 1])
Classifying hour 1 of ./data/Rumble/Testing/PNNN/Sounds/nn08c_20200104_000000.wav torch.Size([5120, 1]), torch.Size([1800000, 1]), torch.Size([5120, 1])
Classifying hour 2 of ./data/Rumble/Testing/PNNN/Sounds/nn08c_20200104_000000.wav torch.Size([5120, 1]), torch.Size([1800000, 1]), torch.Size([5120, 1])
Classifying hour 3 of ./data/Rumble/Testing/PNNN/Sounds/nn08c_20200104_000000.wav torch.Size([5120, 1]), torch.Size([1800000, 1]), torch.Size([5120, 1])
Classifying hour 4 of ./data/Rumble/Testing/PNNN/Sounds/nn08c_20200104_000000.wav torch.Size([5120, 1]), torch.Size([1800000, 1]), torch.Size([5120, 1])
Classifying hour 5 of ./data/Rumble/Testing/PNNN/Sounds/nn08c_20200104_000000.wav torch.Size([5120, 1]), torch.Size([1800000, 1]), torch.Size([5120, 1])
Classifying hour 6 of ./data/Rumble/Testing/PNNN/Sounds/nn08c_20200104_000000.wav tor

In [58]:
continuous_segments = raven_file_helper.find_continuous_segments(scores[:, 1] - scores[:, 0] > 0.5)
long_enough_segments = raven_file_helper.find_long_enough_segments(continuous_segments, n=4)
print(
    f"of the {len(continuous_segments)} segments classified as rumbles ",
    f"only {len(long_enough_segments)} were over a second long.",
)


of the 212 segments classified as rumbles  only 53 were over a second long.


In [59]:
afp = audio_file_processor
segments =  [{'bt':afp.score_index_to_time(bt),'et':afp.score_index_to_time(et)} for bt,et in long_enough_segments]
for s in segments:
    s['duration'] = s['et']-s['bt']
pd.DataFrame(segments)


,bt,et,duration
0,27379.20,27383.04,3.84
1,50005.76,50009.60,3.84
2,50229.12,50235.52,6.40
3,68139.52,68142.08,2.56
4,71465.60,71474.56,8.96
5,71486.08,71491.20,5.12
6,71862.40,71864.96,2.56
7,73356.16,73360.64,4.48
8,73904.64,73907.20,2.56
9,75150.08,75154.56,4.48


In [60]:
from dataclasses import asdict
labels = raven_file_helper.get_all_labels_for_wav_file(audio_file)
import pandas as pd
pd.DataFrame(labels)


,bt,et,lf,hf,duration,audio_file,t1,t2,t3,notes,score,ravenfile
0,50006.2914,50012.4628,16.214,54.500,6.1714,nn08c_20200104_000000.wav,None,None,None,None,None,./data/Rumble/Testing/PNNN/NN_Random_Sample_Ha...
1,51764.2221,51766.8669,27.300,56.400,2.6448,nn08c_20200104_000000.wav,None,None,None,None,None,./data/Rumble/Testing/PNNN/NN_Random_Sample_Ha...
2,63470.5380,63474.7992,19.774,42.700,4.2612,nn08c_20200104_000000.wav,None,None,None,None,None,./data/Rumble/Testing/PNNN/NN_Random_Sample_Ha...
3,73355.8837,73360.8796,17.300,40.000,4.9959,nn08c_20200104_000000.wav,None,None,None,None,None,./data/Rumble/Testing/PNNN/NN_Random_Sample_Ha...
4,73905.0221,73909.2781,21.356,42.711,4.2560,nn08c_20200104_000000.wav,None,None,None,None,None,./data/Rumble/Testing/PNNN/NN_Random_Sample_Ha...
...,...,...,...,...,...,...,...,...,...,...,...,...
92,85942.4897,85947.8772,21.356,71.800,5.3875,nn08c_20200104_000000.wav,None,None,None,None,None,./data/Rumble/Testing/PNNN/NN_Random_Sample_Ha...
93,85947.9879,85950.8173,29.265,51.800,2.8294,nn08c_20200104_000000.wav,None,None,None,None,None,./data/Rumble/Testing/PNNN/NN_Random_Sample_Ha...
94,85960.7554,85967.0047,20.169,70.000,6.2493,nn08c_20200104_000000.wav,None,None,None,None,None,./data/Rumble/Testing/PNNN/NN_Random_Sample_Ha...
95,86023.4856,86025.9948,26.892,54.500,2.5092,nn08c_20200104_000000.wav,None,None,None,None,None,./data/Rumble/Testing/PNNN/NN_Random_Sample_Ha...


In [61]:
audio_file
raven_file_helper.get_all_labels_for_wav_file(audio_file)

[RavenLabel(bt=50006.2914, et=50012.46279999998, lf=16.214, hf=54.5, duration=6.171399999991991, audio_file='nn08c_20200104_000000.wav', t1=None, t2=None, t3=None, notes=None, score=None, ravenfile='./data/Rumble/Testing/PNNN/NN_Random_Sample_Hand_ele.txt'),
 RavenLabel(bt=51764.2221, et=51766.86690000002, lf=27.3, hf=56.4, duration=2.644800000009127, audio_file='nn08c_20200104_000000.wav', t1=None, t2=None, t3=None, notes=None, score=None, ravenfile='./data/Rumble/Testing/PNNN/NN_Random_Sample_Hand_ele.txt'),
 RavenLabel(bt=63470.538, et=63474.79919999995, lf=19.774, hf=42.7, duration=4.261199999949895, audio_file='nn08c_20200104_000000.wav', t1=None, t2=None, t3=None, notes=None, score=None, ravenfile='./data/Rumble/Testing/PNNN/NN_Random_Sample_Hand_ele.txt'),
 RavenLabel(bt=73355.8837, et=73360.87960000004, lf=17.3, hf=40.0, duration=4.995900000038091, audio_file='nn08c_20200104_000000.wav', t1=None, t2=None, t3=None, notes=None, score=None, ravenfile='./data/Rumble/Testing/PNNN/NN